In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print("1")

1


DATASET

In [92]:
class PlayingCardDataset(Dataset):
  def __init__(self, data_dir, transform=None):
    self.data = ImageFolder(data_dir, transform=transform)
  def __len__(self):
    return len(self.data)
  def __getitem__(self, idx):
    return self.data[idx]
  def classes(self):
    return self.data.classes

In [93]:
dataset = PlayingCardDataset(
  data_dir = 'archive/train'
)

In [94]:
data_dir = 'archive/train'
target_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class)

{0: 'ace of clubs', 1: 'ace of diamonds', 2: 'ace of hearts', 3: 'ace of spades', 4: 'eight of clubs', 5: 'eight of diamonds', 6: 'eight of hearts', 7: 'eight of spades', 8: 'five of clubs', 9: 'five of diamonds', 10: 'five of hearts', 11: 'five of spades', 12: 'four of clubs', 13: 'four of diamonds', 14: 'four of hearts', 15: 'four of spades', 16: 'jack of clubs', 17: 'jack of diamonds', 18: 'jack of hearts', 19: 'jack of spades', 20: 'joker', 21: 'king of clubs', 22: 'king of diamonds', 23: 'king of hearts', 24: 'king of spades', 25: 'nine of clubs', 26: 'nine of diamonds', 27: 'nine of hearts', 28: 'nine of spades', 29: 'queen of clubs', 30: 'queen of diamonds', 31: 'queen of hearts', 32: 'queen of spades', 33: 'seven of clubs', 34: 'seven of diamonds', 35: 'seven of hearts', 36: 'seven of spades', 37: 'six of clubs', 38: 'six of diamonds', 39: 'six of hearts', 40: 'six of spades', 41: 'ten of clubs', 42: 'ten of diamonds', 43: 'ten of hearts', 44: 'ten of spades', 45: 'three of clu

DATALOADING

In [95]:
transform = transforms.Compose([
  transforms.Resize((128, 128)),
  transforms.ToTensor()
])

data_dir = 'archive/train'
dataset = PlayingCardDataset(data_dir, transform)

In [96]:
image, label = dataset[100]
image.shape

torch.Size([3, 128, 128])

In [97]:
for image, label in dataset:
  break

In [98]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [99]:
for images, labels in dataloader:
  break

MODEL

In [100]:
class SimpleCardClassifier(nn.Module):
  def __init__(self, num_classes = 53):
    super(SimpleCardClassifier, self).__init__()
    
    self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
    self.features = nn.Sequential(*list(self.base_model.children())[:-1])

    enet_out_size = 1280
    self.classifier = nn.Linear(enet_out_size, num_classes)

  def forward(self, x):
    x = self.features(x)
    output = self.classifier(x)
    return output

In [101]:
model = SimpleCardClassifier(num_classes=53)

In [102]:
model(images)

tensor([[ 0.6378, -0.1952, -0.2676,  ...,  0.1909,  0.4693,  0.2446],
        [ 0.1678, -0.0547,  0.1278,  ..., -0.1037,  0.3094, -0.0679],
        [-0.0081,  0.3479, -0.3992,  ...,  0.0475,  0.0380,  0.0162],
        ...,
        [-0.0488,  0.0768, -0.4368,  ...,  0.4016, -0.0284, -0.5741],
        [ 0.2388, -0.0911, -0.2712,  ..., -0.3123, -0.0832, -0.0626],
        [ 0.0603,  0.0422,  0.4072,  ...,  0.2880, -0.1689,  0.1341]],
       grad_fn=<AddmmBackward0>)

In [103]:
example_out = model(images)
example_out.shape

torch.Size([32, 53])

Training loop

In [104]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [105]:
criterion(example_out, labels)

tensor(4.0386, grad_fn=<NllLossBackward0>)

In [106]:
transform = transforms.Compose([
  transforms.Resize((128,128)),
  transforms.ToTensor()
])

train_folder = 'archive/train/' 
valid_folder = 'archive/valid/'
test_folder = 'archive/test/'

train_dataset = PlayingCardDataset(train_folder, transform=transform)
valid_dataset = PlayingCardDataset(valid_folder, transform=transform)
test_dataset = PlayingCardDataset(test_folder, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
num_epochs = 5
training_losses, val_losses = [], []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = SimpleCardClassifier(num_classes=53)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
  model.train()
  running_loss = 0.0
  for images, labels in train_dataloader:
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item() * labels.size(0)
  training_loss = running_loss / len(train_dataloader.dataset)
  training_losses.append(training_loss)

  model.eval()
  running_loss = 0.0
  with torch.no_grad():
    for images, labels in valid_dataloader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels)
      running_loss += loss.item() * labels.size(0)
  val_loss = running_loss / len(valid_dataloader.dataset)
  val_losses.append(val_loss)

  print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {training_loss}, Valid Loss: {val_loss}")

